# Predicting loan default
#### Business question: 
Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

In [1]:
# import liberaries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load data
loans_2007 = pd.read_csv("..loans_2007.csv")
loans_2007.head()


C:\Users\Baash\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
loans_2007.shape

(42538, 52)

## Data preparation

Since the data set contains 52 features we will devide them into 3 parts 18 varaibles each making it easy to study them closer.

In [4]:
# Making a copy of the dataset
loans_2007_copy = loans_2007
loans_2007_copy.shape

(42538, 52)

In [5]:
# first 18 columns
first_18 = loans_2007_copy.iloc[:,:19]
print(first_18.shape)
first_18.head()

(42538, 19)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other


After analyzing the first 18 columns, i conlcluded that the flowing features need to be removed.

In [6]:
loans_2007 = loans_2007.drop(["id","member_id","funded_amnt","funded_amnt_inv","grade","sub_grade","emp_title","issue_d"], axis = 1)
loans_2007.shape

(42538, 44)

In [7]:
# second 18 columns
second_18 = loans_2007_copy.iloc[:,19:37]
print(second_18.shape)
second_18.head()

(42538, 18)


,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp
0,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00
1,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46
2,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00
3,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00
4,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27


Again after analyzing the second 18 columns, i conlcluded that the flowing features need to be removed.

In [8]:
loans_2007 = loans_2007.drop(["zip_code","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp"], axis = 1)
loans_2007.shape

(42538, 38)

In [9]:
# third 18 columns
third_18 = loans_2007_copy.iloc[:,37:]
print(third_18.shape)
third_18.head()

(42538, 15)


,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


finally, after analyzing the second 18 columns, i conlcluded that the flowing features need to be removed.

In [10]:
loans_2007 = loans_2007.drop(["total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt"], axis = 1)
print(loans_2007.shape)
loans_2007.head()

(42538, 32)


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


* The discarded features either contain redundant information that was not useful for modeling, leaked information from the future or required too much processing to make them useful.


* After removing all those features I was able to reduce the total number of features from 52 to 32.

 For target variable the best candidate feature is loan_status.

In [11]:
# expoliring the candidate target variable
print(loans_2007["loan_status"].value_counts())
print()
print()
print(loans_2007["loan_status"].unique())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


['Fully Paid' 'Charged Off' 'Current' 'In Grace Period'
 'Late (31-120 days)' 'Late (16-30 days)' 'Default' nan
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']


The investors are intrested the outcome of the loan, only 'Fully Paid' and 'Charged Off' values describe the outcome of the loan. The other values describe loans that are still on going. 
Since we're intrested in being able to predict which of these 2 values a loan will fall under we can treat the problem as pinary in which 'Full Paid' = 1 and 'Charged Off' = 0.

In [12]:
# delete all rows that the loan status column is either "Fully Paid" or "Charged Off"
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]
print(loans_2007.shape)
loans_2007["loan_status"].value_counts()

(38770, 32)


Fully Paid     33136
Charged Off     5634
Name: loan_status, dtype: int64

 One other thing to keep in mind is the class inbalance becouse there are 33136 loans that have been fully paid off and only 5634 that were charged off. Class inbalance leads to biased medel.

In [13]:
# Replace the two values(Fully Paid and Charged Off) with 0 and 1
map = {
    "loan_status": {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(map)


In [14]:
loans_2007.shape

(38770, 32)

In [15]:
# remove single value columns that have no predictive value
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)


['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


In [16]:
# Checking the shape of the dataframe
loans_2007.shape

(38770, 23)

In [17]:
# renaming the dataframe to make it easy for further processing
filtered_loans_2007 = loans_2007
filtered_loans_2007.shape
# filtered_loans_2007.head()

(38770, 23)

In [18]:
# checking the number of null values
filtered_loans_2007.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

In [19]:
# drop columns with morethan 50% of values are missing
filtered_loans_2007 = filtered_loans_2007.drop("pub_rec_bankruptcies", axis = 1)
filtered_loans_2007 = filtered_loans_2007.dropna(axis = 0)
filtered_loans_2007.shape

(37675, 22)

In [20]:
#print(filtered_loans_2007.isnull().sum())
#print()
#print()
print(filtered_loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


In [21]:
# changing object datatype columns into integer
object_columns = filtered_loans_2007.select_dtypes(include = ["object"])
object_columns.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


In [22]:
# checking the value counts of categorical columns
#cols = ["home_ownership", "verification_status", "emp_length", "term", "addr_state"]
#for i in cols:
    #print(filtered_loans_2007[i].value_counts())

In [23]:
# checking unique values of purpose and title to decide which one to keep
#print(filtered_loans_2007["purpose"].value_counts())
print()
print()
#print(filtered_loans_2007["title"].value_counts())

* After comparing title and purpose columns i have chose to keep purpose feature. So the title along with last_credit_pull_d, earliest_cr_line, addr_state columns will be droped 

In [24]:
# drop columns
filtered_loans_2007 = filtered_loans_2007.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
filtered_loans_2007.shape

(37675, 18)

In [25]:
# encode the emp_length column also strip % and change to float in int_rate, revol_util columns
map2 = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
filtered_loans_2007["int_rate"] = filtered_loans_2007["int_rate"].str.rstrip("%").astype("float")
filtered_loans_2007["revol_util"] = filtered_loans_2007["revol_util"].str.rstrip("%").astype("float")
filtered_loans_2007 = filtered_loans_2007.replace(map2)
#filtered_loans_2007.head()

In [26]:
# ecoding other categorical columns into dummy variables so we can use them in the model
dummy_df = pd.get_dummies(filtered_loans_2007[["home_ownership","verification_status","purpose","term"]])

filtered_loans_2007 = pd.concat([filtered_loans_2007, dummy_df], axis = 1)

filtered_loans_2007 = filtered_loans_2007.drop(["home_ownership","verification_status","purpose","term"], axis = 1)
print(filtered_loans_2007.shape)
filtered_loans_2007.head()

(37675, 38)


,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,...,0,0,0,0,0,0,0,1,1,0


In [27]:
# class inbalance of target column 
filtered_loans_2007["loan_status"].value_counts()

1    32286
0     5389
Name: loan_status, dtype: int64

we can see that there is class imbalance in target column which can be problematic. This is something to keep in mind while creating a model to predict loans.


#### Data cleaning summery
As i prepared the data, i removed columns that had data leakage issues, contained redundant information, or required additional processing to turn into useful features. i also cleaned features that had formatting issues, and converted categorical columns to dummy variables.

## Data modeling



Sine we are tryping to accurately predict wether a loan applicant will pay off their loan on time or not, it's clear that we are dealing with classification problem.

In [30]:
# assigning cleaned dataframe to new and easy to rememnber variable
loans = filtered_loans_2007 
print(loans.shape)
#print(loans.info())
loans.head()

(37675, 38)


,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,...,0,0,0,0,0,0,0,1,1,0


#### First model:
Logistic regression with k-fold cross validation using sciket_learn 

In [31]:
# import libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

# creating training and target features
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]

# fitting the model
lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv = 3)

# turning predictions into pandas series
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

# printing rates
print(fpr)
print(tpr)

0.9986179664363277
0.9984268484530676


Becouse of the class inbalance of the target variable, you can see that our fpr and tpr are around what we'd expect if the model was predicting all ones. In this situation using penelty for the missclasification of less prevalent class more than other class will help us. To do this instead of using sciket-learns class weight, we can assign the penalty by hand to get a better prediction

In [32]:
#  assigning a harsher penalty manually
penalty = {
    0: 10,
    1: 1
}
lr = LogisticRegression( class_weight = penalty)
predictions = cross_val_predict(lr, features, target, cv = 3)

# turning predictions into a pandas series
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

0.22724580454096743
0.2305977975878343


as you can see assigning harsher penelty significantly lowerd the false postive rate.

#### Second model:
Random forest classifier with k-fold cross validation using sciket_learn

In [33]:
# importing libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict

rf = RandomForestClassifier(class_weight = "balanced", random_state = 1)
predictions = cross_val_predict(rf, features, target, cv = 3)

# turning predictions into a pandas series
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
fpr = fp / (fp + tn)
tpr = tp / (tp + fn)

print(fpr)
print(tpr)

C:\Users\Baash\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9634748272458046
0.9630964866282119


After comparing the Random forest model results to our previous Logistic Regression model, it's clear that the classifier didn't imporve the false positive rate. so for now will stick with the first model